# BOX BOX

## Description:
In a sport 

## Connection
Insert all the value to connect to DB

In [11]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [2]:
txtHost=widgets.Text(
    placeholder ='Insert host',
    description ='Host:',
    disabled = False)

txtUser=widgets.Text(
    placeholder='Insert user',
    description='User',
    disabled = False)

txtPassword=widgets.Text(
    placeholder ='Insert password',
    description ='Password:',
    disabled=False)

txtPort=widgets.Text(
    placeholder ='Insert port',
    description ='Port:',
    disabled=False)

s={'description_width':'initial'}

txtDatabase=widgets.Text(
    style=s,
    placeholder ='Insert database name',
    description ='Database name:',
    disabled=False)

display(txtHost)
display(txtUser)
display(txtPassword)
display(txtPort)
display(txtDatabase)

Text(value='', description='Host:', placeholder='Insert host')

Text(value='', description='User', placeholder='Insert user')

Text(value='', description='Password:', placeholder='Insert password')

Text(value='', description='Port:', placeholder='Insert port')

Text(value='', description='Database name:', placeholder='Insert database name', style=DescriptionStyle(descri…

In [3]:
#established the connection
def connect(b):
    global mydb
    Host=txtHost.value
    User=txtUser.value
    Password=txtPassword.value
    Port=txtPort.value
    Database=txtDatabase.value

    mydb = mysql.connector.connect(
        host=Host,
        user=User,
        password=Password,
        port=Port,
        database=Database
    )
    
buttonconn = widgets.Button(
    description='Connect',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Connect' # (FontAwesome names without the `fa-` prefix)
)

buttonconn.on_click(connect)
display(buttonconn)

Button(description='Connect', style=ButtonStyle(), tooltip='Connect')

###

## Average number of pit stops for Grand Prix

Run the code under, choose the GP you want to calulate and click the button

In [4]:
mgiricursor = mydb.cursor()

mgiricursor.execute(
            'SELECT P.raceId, P.stop, P.lap, D.driverRef, R.name FROM (((pitstops P JOIN drivers D ON P.driverId = D.driverId) JOIN races R ON P.raceId = R.raceId) JOIN circuits C ON R.circuitId = C.circuitId) JOIN results RE ON RE.raceId = R.raceId AND RE.driverId = D.driverId WHERE (R.year BETWEEN 2011 AND 2022) ORDER BY P.raceId, P.stop'
        )
# AND RE.positionOrder BETWEEN 1 AND 5

datalista = mgiricursor.fetchall()

DF = pd.DataFrame(datalista, columns=['Raceid', 'Stop', 'Lap', 'DriverRef', 'GrandPrix'])


#lista GP
GPcursor=mydb.cursor()
GPcursor.execute('SELECT distinct name FROM f1.races where year between 2011 and 2022 order by name')

lista_GP=GPcursor.fetchall()
lista_GP=pd.DataFrame(lista_GP, columns=['GP'])

GrandP=widgets.Dropdown(
    options=lista_GP['GP'],
    description='GrandPrix:',
    disabled=False
)

print('Choose the GP')
display(GrandP)


Choose the GP


Dropdown(description='GrandPrix:', options=('70th Anniversary Grand Prix', 'Abu Dhabi Grand Prix', 'Australian…

In [5]:
output = widgets.Output()

def m_stop(b):
    with output:
        output.clear_output()
        mediastop = DF.groupby(['GrandPrix'])[['Stop']].mean()
        mediastop = mediastop.round(1)
        nameG=GrandP.value
        circuito = mediastop.loc[[nameG]]
        display(circuito)
        
button = widgets.Button(
    description='Calculate',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Calculate',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

button.on_click(m_stop)

display(button)
display(output)

Button(description='Calculate', icon='check', style=ButtonStyle(), tooltip='Calculate')

Output()

###

## Laps to BOXBOX

It takes only the driver who made less than 2 pit stops in a race, excludes drivers who have had problems and have to pit multiple times

In [6]:
LtB = mydb.cursor()

#media giro in cui si fanno i pit stop
LtB.execute(
    'SELECT P.raceId, P.stop, P.lap, D.driverRef, R.name FROM ((pitstops P JOIN drivers D ON P.driverId = D.driverId) JOIN races R ON P.raceId = R.raceId) join circuits C on R.circuitId=C.circuitId where R.year between 2011 and 2022 order by R.raceId, P.stop'
    )

ltb = LtB.fetchall()

DFltb = pd.DataFrame(ltb, columns=['Raceid', 'Stop', 'Lap', 'DriverRef', 'GrandPrix'])

DFltb = DFltb.groupby(['Raceid', 'DriverRef']).filter(lambda x: (x['Stop'].count() <= 2).all())

mean_ltb = DFltb.groupby(['GrandPrix', 'Stop'])[['Lap']].mean()
mean_ltb = mean_ltb.round(1)

GPltb=widgets.Dropdown(
    options=lista_GP['GP'],
    description='GrandPrix:',
    disabled=False
)

print('Choose the GP')
display(GPltb)



Choose the GP


Dropdown(description='GrandPrix:', options=('70th Anniversary Grand Prix', 'Abu Dhabi Grand Prix', 'Australian…

In [7]:
outputltb = widgets.Output()

def laps_to_boxbox(b):
    with outputltb:
        outputltb.clear_output()
        mean_ltb = DFltb.groupby(['GrandPrix', 'Stop'])[['Lap']].mean()
        mean_ltb = mean_ltb.round(1)
        
        nameGPltb=GPltb.value
        circuitoltb = mean_ltb.loc[[nameGPltb]]
        display(circuitoltb)
        
buttonltb = widgets.Button(
    description='Calculate',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Calculate',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

buttonltb.on_click(laps_to_boxbox)

display(buttonltb)
display(outputltb)

Button(description='Calculate', icon='check', style=ButtonStyle(), tooltip='Calculate')

Output()

##

## Plot drivers time

In [8]:
GPt = widgets.Dropdown(
    options=lista_GP['GP'], 
    description='GrandPrix:', 
    disabled=False
)

anni=[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

Year = widgets.Dropdown(options= anni, description='Year:')

# crea una HBox e aggiungi i dropdown come figli
hbox = widgets.HBox([Year, GPt])

# visualizza la HBox
display(hbox)


In [9]:
tempicursor = mydb.cursor()

outputdriv = widgets.Output()

#media giro in cui si fanno i pit stop
def datatime(b):
    global multiselect
    global Datatempi
    with outputdriv:
        outputdriv.clear_output()
        tempicursor.execute(
        'SELECT R.year, R.name, D.driverRef, L.lap, L.time, L.milliseconds FROM ((laptimes L JOIN races R ON L.raceId = R.raceId) JOIN circuits C ON R.circuitId = C.circuitId) JOIN drivers D ON D.driverId = L.driverId WHERE year BETWEEN 2011 AND 2022 and R.year=' + str(Year.value) + ' and R.name= "' + GPt.value + '"'
        )

        listatempi=tempicursor.fetchall()

        Datatempi = pd.DataFrame(listatempi, columns=['Year', 'GrandPrix', 'Driver', 'Lap', 'Time', 'Milli'])
        
        #lista piloti
        drivers=Datatempi['Driver'].unique()
        drivers=sorted(drivers)
        
        multiselect = widgets.SelectMultiple(options=drivers, description = 'Drivers:', rows=len(drivers))
        display(multiselect)
             
buttondriv = widgets.Button(
    description='Drivers',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Drivers')

buttondriv.on_click(datatime)

display(buttondriv)
display(outputdriv)

Button(description='Drivers', style=ButtonStyle(), tooltip='Drivers')

Output()

In [10]:
outputplot = widgets.Output()

def plot_drivers(b):
    with outputplot:
        outputplot.clear_output()
        
        Q3 = Datatempi['Milli'].quantile(0.75)
        Q1 = Datatempi['Milli'].quantile(0.25)
        IQR = Q3 - Q1
        upper_limit = Q3 + 1.5*IQR

        assey = range(((min(Datatempi['Milli'])//10000)*10000), max(Datatempi['Milli']), 200)

        times = [pd.to_datetime(m, unit='ms').time() for m in assey]
        y_labels = [t.strftime('%M:%S.%f')[:-3] for t in times]

        plt.figure(figsize=(20, 10))

        for pilota in multiselect.value:
            # selezionare i dati solo per il pilota corrente
            dati_pilota = Datatempi[Datatempi['Driver'] == pilota]
            # plottare i dati del pilota corrente
            plt.plot(dati_pilota['Lap'], dati_pilota['Milli'], label=pilota)

    
        giri_tick = range(0, max(Datatempi['Lap']), 2)
        plt.xticks(giri_tick)
        plt.yticks(assey, y_labels)

        plt.ylim(min(Datatempi['Milli']), upper_limit)
        plt.xlim(0, max(Datatempi['Lap']))

        plt.xlabel('Lap')
        plt.ylabel('Time (MM:SS.mmm)')

        plt.grid(True)

        # aggiungere una legenda
        plt.legend()

        plt.gca().set_facecolor('lightgrey')

        # mostrare il grafico
        plt.show()

buttonplot = widgets.Button(
    description='Plot drivers',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Drivers'
)

buttonplot.on_click(plot_drivers)

display(buttonplot)
display(outputplot)

Button(description='Plot drivers', style=ButtonStyle(), tooltip='Drivers')

Output()